In [1]:
from __future__ import division
from IPython.display import display
from sympy import Matrix
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')
import sympy as sym
from sympy import *


In [10]:

n = 5 # i = 0..4
g = 9.82
g0 = Matrix([[0], [0], [-g]])
"initialization of arrays for frequently use matrices"
TENSORs = [-1 for j in range(n + 1)]
Ts = [[-1 for j in range(n + 1)] for i in range(n + 1)]
Rs = [[-1 for j in range(n + 1)] for i in range(n + 1)]
Zs = [-1 for j in range(n + 1)]

# jacobians
O = Matrix([0, 0, 0])
Jv = [[O for j in range(n)] for i in range(n)]
Jomega = [[O for j in range(n)] for i in range(n)]

# START Q BLOCK
# FIRST alternative way for "q" as just symbols
# q = symbols("q_1 q_2 q_3 q_4 q_5")
# dq = symbols('\dot{q_1} \dot{q_2} \dot{q_3} \dot{q_4} \dot{q_5}')
# ddq = symbols('\ddot{q_1} \ddot{q_2} \ddot{q_3} \ddot{q_4} \ddot{q_5}')

# SECOND alternative way for "q" as functions q_i(t)
t = Symbol('t')
q = list()
dq = list()
ddq = list()
for i in range(0, n):
    q.append(Function('q_' + str(i + 1))(t))
    dq.append(diff(q[i], t))
    ddq.append(diff(q[i], t, 2))
# END Q BLOCK

# DH parameters 0..4
a = symbols("a_1 a_2 a_3 a_4 a_5")
d = symbols("d_1 d_2 d_3 d_4 d_5")
ai = [a[0], a[1], a[2], 0, 0]
di = [d[0], 0, 0, 0, d[4]]
alphai = [pi / 2, 0, 0, pi / 2, 0]
thi = [ pi * (169./180.) - q[0],
        pi * (65./180.) + (pi / 2.) - q[1],
        -pi * (146./180.)  - q[2],
        pi * (102.5/180) + (pi/2.) - q[3],
        pi * (167.5/180.) - q[4]]

# init mass 0..4
m = symbols('m_1 m_2 m_3 m_4 m_5')
"ALTERNATIVE SYMBOLIC VARIABLES FOR EASIER CODE GENERATION. USING INSTEAD WHICH ARE ABOVE"
xc = symbols('xc1 xc2 xc3 xc4 xc5')
yc = symbols('yc1 yc2 yc3 yc4 yc5')
zc = symbols('zc1 zc2 zc3 zc4 zc5')
x = symbols('x0 x1 x2 x3 x4 x5')
y = symbols('y0 y1 y2 y3 y4 y5')
z = symbols('z0 z1 z2 z3 z4 z6')
Ixx = symbols('I1xx I2xx I3xx I4xx I5xx')
Iyy = symbols('I1yy I2yy I3yy I4yy I5yy')
Izz = symbols('I1zz I2zz I3zz I4zz I5zz')
Ixy = symbols('I1xy I2xy I3xy I4xy I5xy')
Ixz = symbols('I1xz I2xz I3xz I4xz I5xz')
Iyz = symbols('I1yz I2yz I3yz I4yz I5yz')


# MATRICES and additional stuff-
def mySimple(expr):
    "how simpify"
    expr = simplify(expand(expr))
    return expr


def T(i, j, simp=False):
    """
        T(0,1) is equal to {}^0 A_1
    """
    if Ts[i][j] != -1:
        return Ts[i][j]
    else:
        H = eye(4)
        for k in range(i, j):
            Rz = Matrix(
                [[cos(thi[k]), -sin(thi[k]), 0, 0],
                 [sin(thi[k]), cos(thi[k]), 0, 0],
                 [0, 0, 1, 0], [0, 0, 0, 1]])
            Rx = Matrix([[1, 0, 0, 0],
                         [0, cos(alphai[k]), -sin(alphai[k]), 0],
                         [0, sin(alphai[k]), cos(alphai[k]), 0],
                         [0, 0, 0, 1]])
            Tz = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, di[k]], [0, 0, 0, 1]])
            Tx = Matrix([[1, 0, 0, ai[k]], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
            dT = Rz * Tz * Tx * Rx
            H = H * dT
        if simp:
            H = mySimple(H)
        Ts[i][j] = H
        return H


def R(i, j, simp=False):
    "rotation matrices, i>j"
    if Rs[i][j] != -1:
        return Rs[i][j]
    else:
        r = T(i, j)[:3, :3]
        if simp:
            r = mySimple(r)
        Rs[i][j] = r
        return r


def get_z(i, simp=True):
    """
        get_z(1) = z^0_1
    """
    if Zs[i] != -1:
        return Zs[i]
    else:
        z = R(0, i) * Matrix([[0], [0], [1]])
        Zs[i] = z
        return z


def get_r0_0To(i):
    """
        get_r0_0To(1) = r^0_{0,1}
    """
    r = T(0, i)[:3, 3]
    return r  # 3x1


def get_ri_0To(i, simp=False):
    """
        get_ri_0To(1) = r^1_{0, 1}
    """
    r = transpose(R(0, i)) * get_r0_0To(i)
    if simp:
        r = mySimple(r)
    return r  # 3x1


def get_g(i, simp=False):
    """
        get_g(1) = g_1
    """
    g = transpose(R(0, i)) * g0
    if simp:
        g = mySimple(g)
    return g  # 3x1


# INIT JACOBIANS
# for v
for i in range(0, n):
    for j in range(0, i + 1):
        el = get_z(j).cross(get_r0_0To(i+1) - get_r0_0To(j))
        Jv[i][j] = el


def getJv(i):
    """
        getJv(0) = J_{v1}
    """
    j = Matrix(Jv[i])
    j = j.reshape(5, 3)
    return transpose(j)

# for omega
for i in range(0, n):
    for j in range(0, i + 1):
        el = get_z(j)
        Jomega[i][j] = el


def getJomega(i):
    """
        getJomega(0) = J_{omega1}
    """
    j = Matrix(Jomega[i])
    j = j.reshape(5, 3)
    return transpose(j)


def get_v0(i, simp=False):
    """
        get_v0(1) = v^0_1
    """
    v = getJv(i-1) * Matrix(dq)
    return v


def get_vi(i, simp=False):
    """
        get_vi(1) = v^1_1
    """
    v = transpose(R(0, i)) * get_v0(i)
    if simp:
        v = mySimple(v)
    return v  # 3x1


def get_omega0(i, simp=False):
    """
        get_omega0(1) = omega^0_1
    """
    omega = getJomega(i-1) * Matrix(dq)
    return omega


def get_omegai(i, simp=False):
    """
        get_omegai(1) = omega^1_1
    """
    omega = transpose(R(0, i)) * get_omega0(i)
    if simp:
        omega = mySimple(omega)
    return omega  # 3x1


In [6]:
def getTensor(i):
    """ getTensor(0) = I_1"""
    return Matrix([[Ixx[i], Ixy[i], Ixz[i]],
                    [Ixy[i], Iyy[i], Iyz[i]],
                    [Ixz[i], Iyz[i], Izz[i]]])

def get_riici(i):
    """ get_riici(0) = rc_1 """
    return R(0, i) * Matrix([xc[i], yc[i], zc[i]])

def getU():
    """ potential energy """
    U = 0
    for i in range(n):
        U += (m[i] * get_g(i).T * get_ri_0To(i) + get_g(i).T * (m[i] * get_riici(i)))[0]
        
    return U


In [7]:
Jx = - (getJv(i)[2,:]).T * getJomega(i)[1,:] + (getJv(i)[1,:]).T * getJomega(i)[2,:]
Jy =   (getJv(i)[2,:]).T * getJomega(i)[0,:] - (getJv(i)[0,:]).T * getJomega(i)[2,:]
Jz = - (getJv(i)[1,:]).T * getJomega(i)[0,:] + (getJv(i)[0,:]).T * getJomega(i)[1,:]

def getD():
    # inertia matrix
    D = Matrix([[0 for i in range(n)] for j in range(n)])
    for i in range(n):
        D += m[i] * getJv(i).T * getJv(i) + getJomega(i).T * R(0, i) * getTensor(i) * R(0, i).T * getJomega(i) + 2 * (m[i] * get_riici(i))[0] * Jx + 2 * (m[i] * get_riici(i))[1] * Jy + 2 * (m[i] * get_riici(i))[2] * Jz
    return D

In [36]:
def getC(D):
    # Coriolis and Centrifugal matrix
    C = [[[0 for k in range(n)] for j in range(n)] for i in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j][k] = i#(1/2) * (diff(D[k,j], q[i]) + diff(D[k,i], q[j]) - diff(D[i,j], q[k]))

    C_new = [[0 for k in range(n)] for j in range(n)]
    for k in range(n):
        for j in range(n):
            s = 0
            for i in range(n):
                s += C[i][j][k] * q[i]
            C_new[k][j] = s
    return C_new

In [23]:
def getG(U):
    """ gravity matrix"""
    G = [0 for j in range(n)]
    for j in range(n):
        G[j] = diff(U, q[j])
    return G

In [26]:
import numpy
T(0,3)[0].subs([(pi, numpy.pi), (q[0], 0)])

0.981627183447664⋅sin(q₂(t) - 2.70526034059121)⋅sin(q₃(t) + 2.54818070791172) 
- 0.981627183447664⋅cos(q₂(t) - 2.70526034059121)⋅cos(q₃(t) + 2.54818070791172
)

In [79]:
try:
    c[0][0][0]
except TypeError:
    print(1)

1
